<a href="https://colab.research.google.com/github/kheetapatt/nlp-rag-chatbot/blob/main/cactus_chatbot_source_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 2 RAG with LLM Chatbot**

**install**

In [ ]:
!pip install "transformers[torch]" "huggingface_hub[inference]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 15.5 MB/s eta 0:00:00


In [ ]:
!pip install llama-index
!pip install gradio
!pip install einops
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 24.1 MB/s 

In [ ]:
!pip install llama-index-llms-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.26.2
    Uninstalling huggingface-hub-0.26.2:
      Successfully uninstalled huggingface-hub-0.26.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.5.0 requires huggingface-hub>=0.25.1, but you have huggingface-hub 0.23.5 which is incompatible.


In [ ]:
!pip install llama-index-embeddings-huggingface

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 61.1 MB/s eta 0:00:00


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, Document
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

**data preparing**

In [ ]:
import pymupdf

def read_doc(path) :
  doc = pymupdf.open(path)
  data = ""
  for page in doc:
    text = page.get_text()
    data += text
    data += "\n"
  return data

data = []
data.append(read_doc("/content/data/disease_cactus.pdf"))
data.append(read_doc("/content/data/growing_cactus.pdf"))
data.append(read_doc("/content/data/propagation_cactus.pdf"))
data.append(read_doc("/content/data/type_cactus.pdf"))
data

['กระบองเพชรรากเสียรากตายแก้ได้ง่ายๆได้รากใหม่ต้นกลับมาแข็งแรงเหมือนเดิม\nบ่อยครั้งที่เรารับต้นใหม่เข้าบ้านทำให้เราไม่รู้ประวัติของต้นก่อนจะมาถึงเราและไม่ทราบสาเหตุที่ทำให้\nเรานำไม้มาปลูกแล้วเกิดปัญหาต้นไม่โตรากไม่เดินหรือแย่สุดคือรากเน่าจนกินเข้ามาจนถึงแกนต้น\nและเน่าลามจนเกิดเยี่ยวยา\nแต่ข้อสันนิษฐานคือให้สังเกตไม้ก่อนนำเข้าบ้านว่ามีลักษณะสมบูรณ์หรือไม่รากยังขาวแข็งแรงไม่\nเหี่ยวแห้งต้นไม่มีอาการนิ่มหรือฝ่อบริเวณโคนเพราะอาการเหล่านี้สามารถบ่งชี้ถึงสภาพแวดล้อมหรือ\nการเลี้ยงก่อนหน้านี้ได้ว่าอาจจะได้รับน้ำไม่เพียงพอหรือถ้าเป็นไม้ที่ถอดรากมาจะไม่เห็นสภาพของดิน\nเดิมก็อาจจะทำให้ยากต่อการประเมินอาจจะต้องใช้วิธีทดลองปลูกก่อนและถ้าในระยะเวลา\nหลังจากปลูกราว2 สัปดาห์แล้วต้นยังไม่มีการฟื้นตัวใดๆยอดไม่เดินและเมื่อรื้อรากมาดูรากก็ไม่เดิน\nหรืองอกใหม่อาจจะตั้งข้อสันนิษฐานได้ว่ารากเก่านั้นอาจจะมีปัญหาไม่ว่าจากการขาดน้ำมานานหรือ\nโดนโรคและแมลงรบกวน\nวิธีแก้แบบที่จะสามารถลดอัตราการตายหรือสูญเสียได้อีก1 วิธีคือการตัดรากเดิมทิ้งทั้งหมดและ\nกระตุ้นให้เขาสร้างรากใหม่ขึ้นมาทดแทน\nซึ่งในการเปลี่ยนกระถาง

**RAG**

In [ ]:
documents = [Document(text=t) for t in data]

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-large"
)

Settings.chunk_size = 256
Settings.chunk_overlap = 10

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
from llama_index.core import PromptTemplate

system_prompt = "<|SYSTEM|># You are a Q&A assistant. Your goal is to use the information and context provided to create the most natural speech possible."

query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

**integration with LLM**

In [ ]:
from huggingface_hub import  notebook_login
notebook_login()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct"
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=512,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-3.2-3B-Instruct",
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm
Settings.chunk_size = 512

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
index = VectorStoreIndex.from_documents(documents)

**querying**

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
def predict(input, history):
  response = query_engine.query(input)
  return str(response)

In [ ]:
import gradio as gr

gr.ChatInterface(predict).launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98b442a5eb099b0b85.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
